In [3]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
import requests, zipfile, io
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# ------------------------------------------------------------------
# Resolve paths relative to script location (works in Jupyter & .py)
# ------------------------------------------------------------------
try:
    SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    SCRIPT_DIR = os.getcwd()

DATA_DIR = os.path.join(SCRIPT_DIR, "data")
os.makedirs(DATA_DIR, exist_ok=True)

# ------------------------------------------------------------------
# 1. Download Saxony boundary (from GADM level 1 = states)
# ------------------------------------------------------------------
url = "https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_DEU_shp.zip"
r = requests.get(url)

with zipfile.ZipFile(io.BytesIO(r.content)) as z:
    z.extractall(os.path.join(SCRIPT_DIR, "gadm_germany"))

# Read GADM Level 1 (states)
gdf = gpd.read_file(os.path.join(SCRIPT_DIR, "gadm_germany/gadm41_DEU_1.shp"))
saxony = gdf[gdf["NAME_1"] == "Sachsen"]

# ------------------------------------------------------------------
# 2. Function to clip raster with Saxony boundary
# ------------------------------------------------------------------
def clip_raster(input_tif, output_tif, boundary):
    with rasterio.open(input_tif) as src:
        boundary = boundary.to_crs(src.crs)  # match CRS
        out_image, out_transform = mask(src, boundary.geometry, crop=True)
        out_meta = src.meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })

    with rasterio.open(output_tif, "w", **out_meta) as dest:
        dest.write(out_image)
    
    return output_tif

# ------------------------------------------------------------------
# 3. Clip Hansen rasters to Saxony
# ------------------------------------------------------------------
loss_tif = os.path.join(DATA_DIR, "Hansen_GFC-2024-v1.12_lossyear_60N_010E.tif")
treecover_tif = os.path.join(DATA_DIR, "Hansen_GFC-2024-v1.12_treecover2000_60N_010E.tif")

loss_saxony = clip_raster(loss_tif, os.path.join(SCRIPT_DIR, "hansen_lossyear_saxony.tif"), saxony)
treecover_saxony = clip_raster(treecover_tif, os.path.join(SCRIPT_DIR, "hansen_treecover2000_saxony.tif"), saxony)

print("Clipped rasters saved:", loss_saxony, treecover_saxony)

# ------------------------------------------------------------------
# 4. Compute statistics
# ------------------------------------------------------------------
# Pixel size (validation)
with rasterio.open(loss_saxony) as src:
    res_x, res_y = src.res
    pixel_area_ha = abs(res_x * res_y) / 10000
    loss = src.read(1)

loss = loss.flatten()
loss = loss[loss > 0]  # exclude no data

unique, counts = np.unique(loss, return_counts=True)
years = unique.astype(int) + 2000

loss_stats = pd.DataFrame({
    "year": years,
    "pixels": counts
})
loss_stats["area_ha"] = loss_stats["pixels"] * pixel_area_ha

# ------------------------------------------------------------------
# 5. Forest baseline (treecover 2000, threshold 30%)
# ------------------------------------------------------------------
with rasterio.open(treecover_saxony) as src:
    cover2000 = src.read(1)

forest_mask = cover2000 >= 30
forest_area_total = forest_mask.sum() * pixel_area_ha
print(f"Total forest area in 2000 (>=30% canopy): {forest_area_total:.2f} ha")

# % of baseline forest lost per year
loss_stats["pct_of_2000"] = (loss_stats["area_ha"] / forest_area_total) * 100

# Cumulative
loss_stats["cumulative_ha"] = loss_stats["area_ha"].cumsum()
loss_stats["cumulative_pct"] = loss_stats["pct_of_2000"].cumsum()

# ------------------------------------------------------------------
# 6. Trend analysis (linear regression)
# ------------------------------------------------------------------
X = loss_stats["year"].values.reshape(-1, 1)
y = loss_stats["area_ha"].values
model = LinearRegression().fit(X, y)
slope = model.coef_[0]
print(f"Trend in annual forest loss: {slope:.2f} ha per year")

# ------------------------------------------------------------------
# 7. Save extended stats
# ------------------------------------------------------------------
loss_stats.to_csv(os.path.join(SCRIPT_DIR, "saxony_forest_loss_extended.csv"), index=False)
print("Saved extended statistics to saxony_forest_loss_extended.csv")

# ------------------------------------------------------------------
# 8. Plots
# ------------------------------------------------------------------
# Annual hectares
plt.figure(figsize=(12,6))
plt.bar(loss_stats["year"], loss_stats["area_ha"], color="seagreen")
plt.xlabel("Year")
plt.ylabel("Forest loss (ha)")
plt.title("Annual Forest Loss in Saxony (Hectares)")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.savefig(os.path.join(SCRIPT_DIR, "saxony_forest_loss_hectares.png"), dpi=300, bbox_inches="tight")
plt.close()

# Annual percentage
plt.figure(figsize=(12,6))
plt.bar(loss_stats["year"], loss_stats["pct_of_2000"], color="orange")
plt.xlabel("Year")
plt.ylabel("Forest loss (% of 2000 baseline)")
plt.title("Annual Forest Loss in Saxony (% of 2000 forest cover)")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.savefig(os.path.join(SCRIPT_DIR, "saxony_forest_loss_percentage.png"), dpi=300, bbox_inches="tight")
plt.close()

# Cumulative curve
plt.figure(figsize=(12,6))
plt.plot(loss_stats["year"], loss_stats["cumulative_ha"], marker="o", color="darkred", label="Cumulative loss (ha)")
plt.xlabel("Year")
plt.ylabel("Cumulative forest loss (ha)")
plt.title("Cumulative Forest Loss in Saxony")
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend()
plt.savefig(os.path.join(SCRIPT_DIR, "saxony_forest_loss_cumulative.png"), dpi=300, bbox_inches="tight")
plt.close()

print("Saved plots: hectares, percentage, cumulative")

# ------------------------------------------------------------------
# 9. Spatial visualization
# ------------------------------------------------------------------
with rasterio.open(loss_saxony) as src:
    fig, ax = plt.subplots(figsize=(8,8))
    show(src, ax=ax, cmap="Reds")
    saxony.boundary.plot(ax=ax, edgecolor="black")
    plt.title("Forest Loss Locations in Saxony")
    plt.savefig(os.path.join(SCRIPT_DIR, "saxony_forest_loss_map.png"), dpi=300, bbox_inches="tight")
    plt.close()

print("Saved spatial map: saxony_forest_loss_map.png")


Clipped rasters saved: C:\Users\chris\Projekte\JazztageLabor\Deforestation\notebooks\Saxony\hansen_lossyear_saxony.tif C:\Users\chris\Projekte\JazztageLabor\Deforestation\notebooks\Saxony\hansen_treecover2000_saxony.tif
Total forest area in 2000 (>=30% canopy): 0.00 ha
Trend in annual forest loss: 0.00 ha per year
Saved extended statistics to saxony_forest_loss_extended.csv
Saved plots: hectares, percentage, cumulative
Saved spatial map: saxony_forest_loss_map.png


# Putting It Together (2001–2024)

2001–2006 → relatively stable, mostly management + occasional small storm/beetle events.

2007 → Storm Kyrill → spike in forest loss.

2010–2014 → moderate, background harvesting & beetle outbreaks.

2015 → hot summer, some regional stress.

2017 → Storm Herwart → localized windthrow.

2018–2020 → drought + bark beetle epidemics → huge, sustained increase in forest loss.

2021–2024 → continued salvage logging of dead spruce, some stabilization, but losses still higher than pre-2018.

2022 (exceptional fire year):
• State wildfire statistics report ~784 ha of forest burned in Saxony (excl. federal forest). Most of that area burned in June–July. 
wald.sachsen.de

• Within that, the Saxon Switzerland National Park fire (spreading from the Czech side) burned ~115 ha on the German/Saxony side between 25 Jul – 03 Sep 2022. 
staatsregierung.sachsen.de